## **Práctica 7**
## Traducción con redes neuronales

### Fecha de entrega

5 de mayo 2024 11.59 p.m.

### Actividades

- Explorar los datasets disponibles en el *Shared Task de Open Machine Translation de AmericasNLP 2021*
    - [Datasets](https://github.com/AmericasNLP/americasnlp2021/tree/main/data)
    - [Readme](https://github.com/AmericasNLP/americasnlp2021/tree/main#readme)
- Crear un modelo de traducción neuronal usando OpenNMT-py y siguiendo el pipeline visto en clase
    - 0. Obtención de datos y preprocesamiento
        - Considerar que tiene que entrenar su modelo de tokenization
    - 1. Configuración y entrenamiento del modelo
    - 2. Traducción
    - 3. Evaluación
        - Reportar BLEU
        - Reportar ChrF (medida propuesta para el shared task)
        - Más info: [evaluate.py](https://github.com/AmericasNLP/americasnlp2021/blob/main/evaluate.py)        
- Comparar resultados con [baseline](https://github.com/AmericasNLP/americasnlp2021/tree/main/baseline_system#baseline-results)
- Incluir el archivo `*.translated.desubword`

### Extra

- Investigar porque se propuso la medida ChrF en el Shared Task
    - ¿Como se diferencia de BLEU?
    - ¿Porqué es reelevante utilizar otras medidas de evaluación además de BLEU?

### Exploración de datasets

### Comparación

### Resultados

### Investigación
